# Thomas Fire Analysis
#### Author: Anna Ramji ([@a-ramji](https://github.com/a-ramji), a-ramji@github.io)
- This notebook is part of my Thomas Fire analysis [github repository](https://github.com/a-ramji/thomas-fire)
(https://github.com/a-ramji/thomas-fire)


# About

### Purpose


There are two primary goals of this data analysis and visualization. 
1. Use county-level AQI data to show the **air quality in Santa Barbara County from 2017-2018, highlighting the Thomas Fire**, which occurred in December, 2017.
2. Use Landsat and a shapefile of fire perimeters in California to **create a false color image displaying the fire scar from the 2017 Thomas Fire**

### Highlights of analysis
    - Fetch vector data from an online repository
    - Visualize raster data
    - Time series analysis
    - etc
### Dataset description

#### AQI Data
- The AQI data I'll be using in my analysis is [Air Quality Index (AQI)](https://www.airnow.gov/aqi/aqi-basics/) data from the [US Environmental Protection Agency](https://www.epa.gov) to visualize the impact on the AQI of the 2017 [Thomas Fire](https://en.wikipedia.org/wiki/Thomas_Fire) in Santa Barbara County. 
- More specifically, I'll be importing county-level AQI data for 2017 and 2018

#### Landsat Collection 2 Level-2 data from Landsat 8 
- The second is a simplified collection of bands (red, green, blue, near-infrared and shortwave infrared) from the Landsat Collection 2 Level-2 atmosperically corrected surface reflectance data, collected by the Landsat 8 satellite
- This data that was accessed and pre-processed in Microsoft's Planetary Computer to remove data outside land and coarsen the spatial resolution ([Landsat Collection in MPC](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)) by Professor Carmen Galaz-García ([@carmengg](https://github.com/carmengg) on GitHub)

#### California Shapefile
- The third dataset is A shapefile of fire perimeters in California during 2017. The [complete file can be accessed in the CA state geoportal.](https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-fire-perimeters-all-1/about)



### References to datasets

- AQI Data: https://aqs.epa.gov/aqsweb/airdata (Accessed October, 2023)
- Landsat Data: https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2, pre-processed on Microsoft's Planetary Computer by Dr. Carmen Galaz-García (Accessed November, 2023). This file will by in a data folder in my github repo for this project. 
- CA Shapefile: as mentioned above, the [complete file can be accessed in the CA state geoportal](https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-fire-perimeters-all-1/about). I'll be using a local copy of the shapefile in a hidden data folder in this repo. (Accessed November, 2023)

In [1]:
# before we begin our analysis, let's load in the relevant libraries, packages, and modules:

# importing libraries with standard abbreviations
# basic libraries
import os # like here() in R
import numpy as np
import pandas as pd

# for spatial/raster data etc. 
import xarray as xr
import rioxarray as rioxr
import geopandas as gpd

# for plotting 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches # for creating legends
import matplotlib.lines as mlines # for making our customized marker in the legend
from matplotlib.lines import Line2D  # also for this custom marker 
from matplotlib.markers import MarkerStyle # also for custom markers


from shapely.geometry import Point 
from shapely.geometry import box # for making our bounding box/cropping (optional)

from rasterio.features import rasterize # for rasterizing polygons

pd.set_option('display.max_columns', None) # setting custom full output options for data exploration (optional) 



# Importing Data

### AQI Data for 2017-2018

In [2]:

# reading in 2017 data
aqi_17 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2017.zip')

# reading in 2018 data
aqi_18 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2018.zip')


### (pre-processed) Landsat Data

# Geographical context of data

# Data exploration

# Analysis